# Reconocimiento de señales de tráfico
### Máster Universitario en Ingeniería computacional y matemática - Área de inteligencia artificial
### Antonio González Hidalgo (agonzalezhidalgo@uoc.edu)

Usando una red neuronal convolucional.

Para la correcta funcionamiento de este notebook, el notebook debe de estar estructurado de la siguiente manera:

- ./dataset
- ./dataset/info.csv          Fichero que contiene los nombres de las señales correspondientes.
- ./dataset/train/Images      Conjunto de imágenes que formarán el entrenamiento de la red neuronal.
- ./dataset/test/info.csv.    Fichero que contiene las categórias de las imágenes de test.
- ./dataset/test/Images       Conjunto de imágenes que constituirán el test.

Este notebook se ha ejecutado usando el dataset de _BelgiumTS_
https://btsd.ethz.ch/shareddata/

In [1]:
import import_ipynb
import tfm_generic_functions as traffic

# https://docs.python.org/3/library/os.html
import os

# https://docs.scipy.org/doc/numpy/reference/
import numpy as np

# https://keras.io/models/model/
import keras

from keras import models

# Core Layers: https://keras.io/layers/core/
# Convolution Layers: https://keras.io/layers/convolutional/
from keras import layers

# https://keras.io/preprocessing/image/
from keras.preprocessing.image import ImageDataGenerator

# https://keras.io/callbacks/
from keras.callbacks import ModelCheckpoint

importing Jupyter notebook from tfm_generic_functions.ipynb


Using TensorFlow backend.


In [2]:
# Obtenemos el directorio actual como trabajo.
ROOT_PATH = os.getcwd()

# Establecemos la dimensión de las imágenes.
IMG_SHAPE = (64, 64)

# Configuramos la CNN
EPOCHS = 15
ACTIVATIONS = ['sigmoid', 'linear', 'tanh', 'relu', 'selu', 'elu', 'softplus', 'softsign', 'hard_sigmoid']

print("Tamaño de las imágenes de entrada: ", IMG_SHAPE)
IMG_SHAPE_LEN = IMG_SHAPE[0] * IMG_SHAPE[1]
print("Vectorizando la entrada, sería de un tamaño: ", IMG_SHAPE_LEN)

# Obtenemos los paths de trabajo
labels_path = os.path.join(ROOT_PATH, "dataset_bel/labels.csv")
train_path = os.path.join(ROOT_PATH, "dataset_bel/train")
test_path = os.path.join(ROOT_PATH, "dataset_bel/test")

Tamaño de las imágenes de entrada:  (64, 64)
Vectorizando la entrada, sería de un tamaño:  4096


In [3]:
# Cargamos las imágenes de entrenamiento y de test.
images_train, labels_train = traffic.readDataset(train_path, IMG_SHAPE, False)
images_test, labels_test = traffic.readDataset(test_path, IMG_SHAPE, False)

# Convertimos las listas a array numpy de float32
np_images_train = np.asarray(images_train, dtype = np.float32)
np_labels_train = np.asarray(labels_train, dtype = np.int8)

np_images_test = np.asarray(images_test, dtype = np.float32)
np_labels_test = np.asarray(labels_test, dtype = np.int8)

# Recuperamos los nombres de las categorias. Los diferentes tipo de señales
# que se van a clasificar.
titles = traffic.read_csv(labels_path, ",")

# Se imprime información de los datos cargados.
traffic.print_size_dataset(images_train, labels_train, np_images_train, np_labels_train, "train")
traffic.print_size_dataset(images_test, labels_test, np_images_test, np_labels_test, "test")
print("Titles total: ", len(titles))
traffic.print_signals_attributes(10, images_train, labels_train, titles)

Total images (train):  4575
Total labels (train):  62
Images shape:  (4575, 64, 64, 3)
Labels shape:  (4575,)
Total images (test):  2520
Total labels (test):  53
Images shape:  (2520, 64, 64, 3)
Labels shape:  (2520,)
Titles total:  62
Signal:  A31
shape:  (64, 64, 3) 	min: 0.04844037224264742 	max:  0.9921568627450981
shape:  (64, 64, 3) 	min: 0.012683823529412207 	max:  1.0
shape:  (64, 64, 3) 	min: 0.03416077856924062 	max:  0.7854355755974258
shape:  (64, 64, 3) 	min: 0.0391180300245094 	max:  0.8150242225796563
shape:  (64, 64, 3) 	min: 0.0064950980392159095 	max:  0.996078431372549
shape:  (64, 64, 3) 	min: 0.06834022671568622 	max:  0.8909658394607853
shape:  (64, 64, 3) 	min: 0.024230238970588236 	max:  0.9842294730392157
shape:  (64, 64, 3) 	min: 0.04133013556985346 	max:  1.0
shape:  (64, 64, 3) 	min: 0.023467419194240183 	max:  0.7593352673100491
shape:  (64, 64, 3) 	min: 0.01371017156862504 	max:  0.9921568627450981
shape:  (64, 64, 3) 	min: 0.07595454197303977 	max:  0.756

In [4]:
# Convertimos las labels de manera categórica
labels_categorical_train = keras.utils.to_categorical(np_labels_train)
labels_categorical_test = keras.utils.to_categorical(np_labels_test)

print("Ejemplo primera imagen de manera categórica: ", labels_categorical_train[0])

Ejemplo primera imagen de manera categórica:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [5]:
def get_keras_model(actovation):
    # IMPLEMENTACIÓN RED NEURONAL
    # En Keras la envoltura para cualquier red neuronal se crea con la clase Sequential
    model = models.Sequential()

    model.add(layers.Conv2D(32, (5, 5),
                            activation=actovation, input_shape=(IMG_SHAPE[0], IMG_SHAPE[1], 3)))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.Conv2D(64, (5, 5), activation=actovation))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))
    
    model.add(layers.Conv2D(128, (5, 5), activation=actovation))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.Flatten())
    model.add(layers.Dense(len(set(labels_train)), activation='softmax'))
    return model

In [6]:
for activation in ACTIVATIONS:
    print(activation)
    model = get_keras_model(activation)
    model.compile(loss="categorical_crossentropy",
                     optimizer="sgd",
                     metrics=['accuracy'])
    model.fit(np_images_train, labels_categorical_train,
              batch_size = 32,
              epochs = EPOCHS,
              callbacks=[ModelCheckpoint('model_64_64_bel_' + activation + '.h5', save_best_only = False)])

    test_loss, test_acc = model.evaluate(np_images_test, labels_categorical_test)
    print('Test loss:', test_loss)
    print('Test accuracy:', test_acc)

sigmoid
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/15
4575/4575 [==============================] - 25s 5ms/step - loss: 3.7847 - acc: 0.0603
Epoch 2/15
4575/4575 [==============================] - 24s 5ms/step - loss: 3.6923 - acc: 0.0680
Epoch 3/15
4575/4575 [==============================] - 24s 5ms/step - loss: 3.6735 - acc: 0.0684
Epoch 4/15
4575/4575 [==============================] - 26s 6ms/step - loss: 3.6567 - acc: 0.0737
Epoch 5/15
4575/4575 [==============================] - 25s 6ms/step - loss: 3.6465 - acc: 0.0717
Epoch 6/15
4575/4575 [==============================] - 25s 5ms/step - loss: 3.6338 - acc: 0.0721
Epoch 7/15
4575/4575 [==============================] - 25s 6ms/step - loss: 3.6296 - acc: 0.0756
Epoch 8/15
4575/4575 [==============================] - 25s 6ms/step - 

4575/4575 [==============================] - 54s 12ms/step - loss: 0.3160 - acc: 0.9183
Epoch 10/15
4575/4575 [==============================] - 54s 12ms/step - loss: 0.2636 - acc: 0.9257
Epoch 11/15
4575/4575 [==============================] - 52s 11ms/step - loss: 0.2422 - acc: 0.9307
Epoch 12/15
4575/4575 [==============================] - 53s 12ms/step - loss: 0.1955 - acc: 0.9440
Epoch 13/15
4575/4575 [==============================] - 53s 12ms/step - loss: 0.1660 - acc: 0.9532
Epoch 14/15
4575/4575 [==============================] - 53s 12ms/step - loss: 0.1602 - acc: 0.9513
Epoch 15/15
2520/2520 [==============================] - 8s 3ms/step
Test loss: 0.165745300883455
Test accuracy: 0.955952380952381
elu
Epoch 1/15
4575/4575 [==============================] - 44s 10ms/step - loss: 3.3715 - acc: 0.2138
Epoch 2/15
4575/4575 [==============================] - 43s 9ms/step - loss: 2.4226 - acc: 0.4210
Epoch 3/15
4575/4575 [==============================] - 43s 9ms/step - loss: 1.8